# Tier 3, Week 8: Quantum Encryption (BB84)

**Audience:** Undergraduate / Advanced

**Objective:** Apply the principles of superposition and measurement to cryptography. Formally derive the BB84 protocol, prove its security against simple attacks using the No-Cloning Theorem and Information-Disturbance tradeoff, and understand the classical post-processing steps required to distill a secure key.

## 1. Bridging from Past Weeks

* **Week 1 & 2:** We learned that measuring a superposition state in the "wrong" basis yields a probabilistic, random result.
* **Week 5 (Noise):** We learned that interactions with an environment (like an eavesdropper) cause decoherence and change the state.
* **Week 7 (VQE):** We learned about "classical post-processing" loops.

**Week 8: BB84** uses these principles to solve the **Key Distribution Problem**:

> *How can Alice and Bob agree on a secret random key if they only have a public channel that Eve might be listening to?*

In classical physics, this is impossible without pre-shared secrets (like in RSA). In quantum physics, it is possible because **observation causes disturbance**.

## 2. The BB84 Formalism: Two Bases

BB84 is the first quantum key distribution (QKD) protocol, developed by Charles Bennett and Gilles Brassard in 1984. It allows two parties, we named as Alice and Bob, to securely exchange a cryptographic key that is immune to eavesdropping, thanks to the fundamental principles of quantum mechanics. The BB84 protocol relies on two non-orthogonal bases for a single qubit.

### The Bases

1.  **Rectilinear (Z) Basis:**
    * States: $|0\rangle$ and $|1\rangle$
    * Measurement Observable: $Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$

2.  **Diagonal (X) Basis:**
    * States: $|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$ and $|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$
    * Measurement Observable: $X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$



### Mathematical Properties

The crucial property is that these bases are **mutually unbiased**. If you prepare a state in one basis and measure in the other, the outcome is perfectly random.

* **Prepare Z, Measure X:**
    $$ |\langle + | 0 \rangle|^2 = \left| \frac{1}{\sqrt{2}} \right|^2 = \frac{1}{2} $$
    $$ |\langle - | 0 \rangle|^2 = \left| \frac{1}{\sqrt{2}} \right|^2 = \frac{1}{2} $$
    (Same for $|1\rangle$).

* **Prepare X, Measure Z:**
    $$ |\langle 0 | + \rangle|^2 = \frac{1}{2} $$
    $$ |\langle 1 | + \rangle|^2 = \frac{1}{2} $$

This uncertainty is the shield that protects the key.

## 3. The Protocol Steps

1.  **Alice's Transmission:**
    Alice generates two random classical bit strings:
    * **Data bits (**$a$**):** e.g., `1 0 1 1 ...`
    * **Basis bits (**$b$**):** e.g., `Z X Z X ...` (where 0=Z, 1=X)

    She encodes qubits based on these choices:
    * If $b_i = Z$ and $a_i = 0 \rightarrow$ Send $|0\rangle$
    * If $b_i = Z$ and $a_i = 1 \rightarrow$ Send $|1\rangle$
    * If $b_i = X$ and $a_i = 0 \rightarrow$ Send $|+\rangle$
    * If $b_i = X$ and $a_i = 1 \rightarrow$ Send $|-\rangle$

2.  **Bob's Measurement:**
    Bob **doesn't know** Alice's basis choices yet. He generates his own random string of **Basis bits (**$b'$**):** e.g., `Z Z X X ...`.
    He measures the incoming qubits in the basis specified by $b'$.

3.  **Sifting (The Classical Handshake):**
    * Alice and Bob communicate over a public classical channel. Then, they compare their basis strings ($b$ and $b'$).
    * They **keep** the bits where their bases matched ($b_i = b'_i$) and **discard** the bits where their bases differed ($b_i \neq b'_i$).

    This leaves them with a **Raw Key**. In the absence of noise/Eve, this key would be identical.

#### Sifting Example: Generating the Raw Key

Let's walk through an example of how Alice and Bob sift their data to produce a shared Raw Key.

**1. Transmission Phase (Quantum)**
* **Alice** generates random data bits and random basis choices. She sends qubits to Bob.
* **Bob** randomly chooses bases to measure the incoming qubits.

| Bit Index | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **Alice's Data Bit** | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| **Alice's Basis** | Z | X | Z | X | X | Z | Z |
| **Bob's Basis** | Z | Z | X | X | Z | Z | X |
| **Bob's Measurement** | 1 | 0 | 0 | 1 | 1 | 1 | 1 |

* *Note:* In bit 2, Alice used X and sent $|+\rangle$ (for 0). Bob used Z. He measured 0 by random chance (50%).
* *Note:* In bit 3, Alice used Z and sent $|1\rangle$. Bob used X. He measured 0 ($|+\rangle$) by random chance.

**2. Sifting Phase (Classical Communication)**
Alice and Bob now talk over a public channel. They announce **only** their basis choices.

* **Alice says:** "My bases were: Z, X, Z, X, X, Z, Z"
* **Bob says:** "My bases were: Z, Z, X, X, Z, Z, X"

They compare the lists and find the matches:

| Bit Index | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **Match?** | **YES** | NO | NO | **YES** | NO | **YES** | NO |

**3. Generating the Raw Key**
They both agree to **keep** bits 1, 4, and 6, and **discard** bits 2, 3, 5, and 7.

* **Alice's Raw Key:** `1 1 1` (From indices 1, 4, 6)
* **Bob's Raw Key:** `1 1 1` (From indices 1, 4, 6)

In this ideal example without noise or Eve, their keys match perfectly. If Eve had been listening, some of these matching bits might have been flipped (errors), which they would detect in the next phase (Error Checking).

## 4. Security Proof: Eve's Disturbance

Why can't Eve just intercept the qubit, copy it, and send it to Bob? This is because of the No-Cloning Theorem that happened in Quantum Information.

### No-Cloning Theorem

Eve cannot create a perfect copy of the qubit $|\psi\rangle$ to store for later.

$$ U(|\psi\rangle \otimes |0\rangle_E) \neq |\psi\rangle \otimes |\psi\rangle $$

She must measure it *now* to get information.

### The Intercept-Resend Attack

Suppose Eve tries to intercept the qubit, measure it, and send a new qubit to Bob based on her result.

#### Scenario Example: Eve Intercepts a Single Qubit

Let's trace exactly what happens when Eve attacks a single bit.

1.  **Alice's Action:**
    * Alice chooses **Data bit `0`** and **Basis `Z`**.
    * She sends the state $|\psi\rangle = |0\rangle$.

2.  **Eve's Interception:**
    * Eve intercepts the qubit. She doesn't know the basis, so she randomly chooses to measure in the **X (Diagonal) Basis**.
    * Since $|0\rangle = \frac{1}{\sqrt{2}}(|+\rangle + |-\rangle)$, Eve measures $|+\rangle$ or $|-\rangle$ with 50% probability each.
    * Let's say Eve measures $|+\rangle$. She sends a new qubit in state $|+\rangle$ to Bob.
    * **Damage Done:** The state has collapsed from $|0\rangle$ to $|+\rangle$. Information has been lost.

3.  **Bob's Measurement:**
    * Bob randomly chooses the **Z (Rectilinear) Basis** (matching Alice).
    * He receives Eve's qubit, which is in state $|+\rangle$.
    * He measures $|+\rangle$ in the Z basis. Since $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$, Bob gets:
        * **Result `0`** with 50% probability (Lucky! No error).
        * **Result `1`** with 50% probability (**Error!** Alice sent 0, Bob got 1).

4.  **Sifting & Check:**
    * Alice and Bob announce their bases: "We both used Z!"
    * They keep this bit.
    * Later, they compare a sample of their key. They find a mismatch (Alice has 0, Bob has 1).
    * **Conclusion:** Because they used the same basis but got different results, they know **Eve must have intercepted the qubit.**

* **Case 1: Eve guesses the right basis (50% chance).**
    * Alice sends $|0\rangle$ (Z-basis).
    * Eve measures in Z. She gets `0`. She sends $|0\rangle$ to Bob.
    * Bob measures in Z. He gets `0`.
    * **Result:** No error introduced. Eve knows the bit.

* **Case 2: Eve guesses the wrong basis (50% chance).**
    * Alice sends $|0\rangle$ (Z-basis).
    * Eve measures in X. She gets $|+\rangle$ (randomly). She sends $|+\rangle$ to Bob.
    * Bob measures in Z (the correct basis).
        * Since he receives $|+\rangle$, he will measure `0` with 50% probability or `1` with 50% probability.
    * **Result:** Eve has randomized the state. Even though Bob measured in the correct basis, he has a **50% chance of getting the wrong answer (1 instead of 0)**.

### Calculating the Error Rate

If Alice and Bob compare a subset of their Sifted Key, they can estimate the **Quantum Bit Error Rate (QBER)**.

* Eve attacks $N$ bits.
* She guesses wrong on $N/2$ bits ($50\%$ of guessing wrong).
* On those wrong guesses, Bob gets an error $50\%$ of the time.
* Total Error Rate induced by Eve:
    $$ E = \frac{1}{2}_{(\text{wrong guess from Eve})} \times \frac{1}{2}_{(\text{wrong guess from Bob})} = 25\% $$

If Alice and Bob detect an error rate $\approx 25\%$, they know Eve is listening. In practice, any error rate above $\approx 11\%$ is considered unsafe.

## 5. Classical Post-Processing

Finding a raw key is not enough. Real hardware has noise (generating errors without Eve), and Eve might have partial information (less than 25% error).

### Error Reconciliation (Information Reconciliation)

Alice and Bob must fix the errors in their keys without revealing the key itself.

#### Example: The Cascade Protocol (Step-by-Step)

Here is how they find and fix errors interactively:

1.  **Permutation (Shuffling):**
    * Alice and Bob agree on a random permutation to shuffle their key bits to spread out any "burst" errors randomly across the key.

2.  **Block Splitting & Parity Exchange:**
    * They divide their keys into blocks of a fixed size (e.g., $k$ bits).
    * They calculate the **parity** (sum modulo 2) of each block.
    * Alice sends her parities to Bob.

3.  **Parity Comparison:**
    * **Match:** If parities match (e.g., both 0), they assume no errors (or an even number of errors) and move on.
    * **Mismatch:** If parities differ, they know there is an **odd number of errors** (usually 1) in that block.

4.  **Binary Search (Finding the Error):**
    [Image of binary search parity check diagram]
    * They perform a "Divide and Conquer" search on the mismatched block:
        1.  Split block into Left and Right halves.
        2.  Compare parity of the Left half.
        3.  If mismatch, the error is in the Left half. If match, it's in the Right.
        4.  Repeat until the single erroneous bit is isolated.
        5.  Bob flips that bit to correct it.

5.  **Iteration:**
    * They re-shuffle the entire key with a *new* permutation and repeat the process to catch any remaining even-number errors.

### 5.B. Privacy Amplification

Even after fixing errors, Eve might know, say, 10% of the bits. Alice and Bob want a shorter key that Eve knows *nothing* about.

* **Method:** Universal Hashing.
* They agree on a random hash function $f$ (like XORing random subsets of bits).
* New Key $K_{final} = f(K_{raw})$.
* **Result:** If Eve knew 10 bits of a 100-bit key, and we compress it to 50 bits, Eve's knowledge of the new 50 bits drops exponentially to near zero.

**Final Secure Key Rate Formula:**
$$ R \approx 1 - H_1(e) - H_2(e) $$
Where $H(e)$ is the binary entropy of the error rate.
* 1st $H_1(e)$: Cost of error correction.
* 2nd $H_2(e)$: Bits lost to privacy amplification (assuming Eve knows as much as the noise allows).